Анализ данных с помощью объединений и конкатенации

Набор данных: Используйте набор данных "tips" и набор данных "flights" из Seaborn.

Инструкции:

Загрузите наборы данных "tips" и "flights" в отдельные Polars DataFrames.

Из набора данных 'flights' выберите столбцы 'year', 'month' и 'passengers'.

Создайте новый DataFrame из 'flights', который агрегирует общее количество пассажиров за год.

Объедините эти агрегированные данные с набором данных 'tips' по столбцу 'year' (примечание: для этого вам может понадобиться создать столбец 'year' в 'tips').

Конкатенируйте DataFrame 'tips' (отфильтрованный по времени 'Dinner') с первыми 50 строками 'flights' (подумайте, каким образом).
Отобразите первые 10 строк объединенного и конкатенированного DataFrame.



In [1]:
import seaborn as sns
import polars as pl

In [3]:
#Загрузите наборы данных "tips" и "flights" в отдельные Polars DataFrames.
tips_df = pl.from_pandas(sns.load_dataset('tips'))
flights_df = pl.from_pandas(sns.load_dataset('flights'))

In [4]:
#Из набора данных 'flights' выберите столбцы 'year', 'month' и 'passengers'.
flights_df.select(['year','month','passengers'])

year,month,passengers
i64,cat,i64
1949,"""Jan""",112
1949,"""Feb""",118
1949,"""Mar""",132
1949,"""Apr""",129
1949,"""May""",121
…,…,…
1960,"""Aug""",606
1960,"""Sep""",508
1960,"""Oct""",461


year,passengers
i64,i64
1956,3939
1959,5140
1949,1520
1958,4572
1952,2364
…,…
1950,1676
1955,3408
1954,2867


In [13]:
#Создайте новый DataFrame из 'flights', который агрегирует общее количество пассажиров за год.
flights_agg = (flights_df
              .group_by('year')
              .agg(pl.col('passengers').sum().alias('total_passengers')))

In [14]:
flights_agg

year,total_passengers
i64,i64
1949,1520
1960,5714
1959,5140
1952,2364
1956,3939
…,…
1955,3408
1951,2042
1957,4421


In [21]:
#Объедините эти агрегированные данные с набором данных 'tips' по столбцу 'year' (примечание: для этого вам может понадобиться создать столбец 'year' в 'tips').
tips_df = tips_df.with_columns(pl.lit(2016).cast(pl.Int64).alias('year'))

In [22]:
joined_df = tips_df.join(flights_agg, on='year', how='left')

In [31]:
print(joined_df.head(10))

shape: (10, 9)
┌────────────┬──────┬────────┬────────┬───┬────────┬──────┬──────┬──────────────────┐
│ total_bill ┆ tip  ┆ sex    ┆ smoker ┆ … ┆ time   ┆ size ┆ year ┆ total_passengers │
│ ---        ┆ ---  ┆ ---    ┆ ---    ┆   ┆ ---    ┆ ---  ┆ ---  ┆ ---              │
│ f64        ┆ f64  ┆ cat    ┆ cat    ┆   ┆ cat    ┆ i64  ┆ i64  ┆ i64              │
╞════════════╪══════╪════════╪════════╪═══╪════════╪══════╪══════╪══════════════════╡
│ 16.99      ┆ 1.01 ┆ Female ┆ No     ┆ … ┆ Dinner ┆ 2    ┆ 2016 ┆ null             │
│ 10.34      ┆ 1.66 ┆ Male   ┆ No     ┆ … ┆ Dinner ┆ 3    ┆ 2016 ┆ null             │
│ 21.01      ┆ 3.5  ┆ Male   ┆ No     ┆ … ┆ Dinner ┆ 3    ┆ 2016 ┆ null             │
│ 23.68      ┆ 3.31 ┆ Male   ┆ No     ┆ … ┆ Dinner ┆ 2    ┆ 2016 ┆ null             │
│ 24.59      ┆ 3.61 ┆ Female ┆ No     ┆ … ┆ Dinner ┆ 4    ┆ 2016 ┆ null             │
│ 25.29      ┆ 4.71 ┆ Male   ┆ No     ┆ … ┆ Dinner ┆ 4    ┆ 2016 ┆ null             │
│ 8.77       ┆ 2.0  ┆ Male   ┆ No     ┆

In [29]:
print(pl.concat(
    [tips_df.filter(pl.col('time')=='Dinner'),
    flights_df[0:50]],
    how='diagonal'
).head(10))

shape: (10, 10)
┌────────────┬──────┬────────┬────────┬───┬──────┬──────┬───────┬────────────┐
│ total_bill ┆ tip  ┆ sex    ┆ smoker ┆ … ┆ size ┆ year ┆ month ┆ passengers │
│ ---        ┆ ---  ┆ ---    ┆ ---    ┆   ┆ ---  ┆ ---  ┆ ---   ┆ ---        │
│ f64        ┆ f64  ┆ cat    ┆ cat    ┆   ┆ i64  ┆ i64  ┆ cat   ┆ i64        │
╞════════════╪══════╪════════╪════════╪═══╪══════╪══════╪═══════╪════════════╡
│ 16.99      ┆ 1.01 ┆ Female ┆ No     ┆ … ┆ 2    ┆ 2016 ┆ null  ┆ null       │
│ 10.34      ┆ 1.66 ┆ Male   ┆ No     ┆ … ┆ 3    ┆ 2016 ┆ null  ┆ null       │
│ 21.01      ┆ 3.5  ┆ Male   ┆ No     ┆ … ┆ 3    ┆ 2016 ┆ null  ┆ null       │
│ 23.68      ┆ 3.31 ┆ Male   ┆ No     ┆ … ┆ 2    ┆ 2016 ┆ null  ┆ null       │
│ 24.59      ┆ 3.61 ┆ Female ┆ No     ┆ … ┆ 4    ┆ 2016 ┆ null  ┆ null       │
│ 25.29      ┆ 4.71 ┆ Male   ┆ No     ┆ … ┆ 4    ┆ 2016 ┆ null  ┆ null       │
│ 8.77       ┆ 2.0  ┆ Male   ┆ No     ┆ … ┆ 2    ┆ 2016 ┆ null  ┆ null       │
│ 26.88      ┆ 3.12 ┆ Male   ┆ No   

/var/folders/2w/zv9tyjqd4n1cxm711skjgh9c0000gn/T/ipykernel_60566/1731658712.py:1: CategoricalRemappingWarning: Local categoricals have different encodings, expensive re-encoding is done to perform this merge operation. Consider using a StringCache or an Enum type if the categories are known in advance
  print(pl.concat(
